In [1]:
# %pip install -r requirements.txt
%pip install underthesea==1.2.2a0

                                              0.0/7.5 MB ? eta -:--:--
                                              0.1/7.5 MB 1.9 MB/s eta 0:00:04
     -                                        0.2/7.5 MB 2.5 MB/s eta 0:00:03
     --                                       0.4/7.5 MB 3.2 MB/s eta 0:00:03
     ---                                      0.7/7.5 MB 4.2 MB/s eta 0:00:02
     -----                                    1.0/7.5 MB 5.1 MB/s eta 0:00:02
     -----                                    1.0/7.5 MB 5.1 MB/s eta 0:00:02
     -----                                    1.0/7.5 MB 5.1 MB/s eta 0:00:02
     -----                                    1.0/7.5 MB 5.1 MB/s eta 0:00:02
     -----                                    1.0/7.5 MB 5.1 MB/s eta 0:00:02
     -----                                    1.0/7.5 MB 5.1 MB/s eta 0:00:02
     -----                                    1.0/7.5 MB 5.1 MB/s eta 0:00:02
     -----                                    1.0/7.5 MB 5.1 MB/s eta 0

  error: subprocess-exited-with-error
  
  Preparing metadata (pyproject.toml) did not run successfully.
  exit code: 1
  
  [26 lines of output]
  <string>:12: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  Partial import of sklearn during the build process.
  Traceback (most recent call last):
    File "<string>", line 187, in get_numpy_status
  ModuleNotFoundError: No module named 'numpy'
  Traceback (most recent call last):
    File "D:\jupyter_proj\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      main()
    File "D:\jupyter_proj\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "D:\jupyter_proj\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", lin

In [1]:
import asyncio
import base64
import decimal
import json
from datetime import datetime, date
import os
import spark

import flask
# import pymysql
import requests
from flask import Flask, render_template, request
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVC
from sklearn import tree
from sklearn.metrics import accuracy_score, classification_report
import openpyxl
import pandas as pd
import numpy as np
import preprocess_kgptalkie as ps
import re
from flask_cors import CORS
import glob
import os
from underthesea import sentiment, pos_tag, word_tokenize
from flask import g
from sklearn.pipeline import Pipeline
import underthesea
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from pandas.errors import ParserError
import csv


SIZE_DEMO = 5000

root = '/jupyter_proj/dataset/**/'
# print(all_files)


def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    # x = str(x).lower()
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    # x = ps.remove_accented_chars(x)
    # x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

def load_data_train_to_excel(root):
    data_train = pd.DataFrame()
    all_files_train = glob.glob(root + "train/**/*.txt", recursive=True)
    for f in all_files_train:     
        df = pd.read_csv(f, header=None, sep = ' ', names = ['Review','Sentiment'], on_bad_lines='skip')
        with open(f, mode='r', encoding="utf8") as f:
            df['Review'] = get_clean(f.read())
            df['Sentiment'] = ""
        data_train = pd.concat([data_train, df], ignore_index=True) 
        # print(data)
    data_train.to_csv('data_train.csv', index=False)

# i=-1
# all_files_test = glob.glob(f'{root}test/{i}/*.txt', recursive=True)
# for f in all_files_test:     
#     df = pd.read_csv(f, header=None, sep = ' ', names = ['Review','Sentiment'], on_bad_lines='skip')
#     with open(f, mode='r', encoding="utf8") as f:
#         df['Review'] = get_clean(f.read())
#         df['Sentiment'] = "negative"
#     data_test = pd.concat([data_test, df], ignore_index=True) 
#     print(data_test)


def load_data_test_to_excel(root):
    data_test = pd.DataFrame()
    sentiment_array = ["negative", "positive", "neutral"]
    all_files_test = []
    for i in [-1,1,2]:
        all_files_test = glob.glob(f'{root}test/{i}/*.txt', recursive=True)
        print(i)
        for f in all_files_test:
            df = pd.read_csv(f, header=None, sep = ' ', names = ['Review','Sentiment'], on_bad_lines='skip', quoting=csv.QUOTE_NONE)
            with open(f, mode='r', encoding="utf8") as f:
                df['Review'] = get_clean(f.read())
                df['Sentiment'] = sentiment_array[(i+1) if i==-1 else i]
            data_test = pd.concat([data_test, df], ignore_index=True) 
            # print(data_test
    data_test.to_csv('data_test.csv', index=False)


# load_data_train_to_excel(root)
# load_data_test_to_excel(root)

df = pd.read_csv('data_test.csv')
df_train = pd.read_csv('data_train.csv')


# # existing_file = 'Book2.csv'
# # df = pd.read_csv(existing_file)
# # df['Review'] = df['Review'].apply(lambda x: get_clean(x))

# # total_files = len(df)
# # train_size = int(0.6 * total_files)
# # val_size = int(0.2 * total_files)
# # test_size = total_files - train_size - val_size

# # train_files = df[:train_size]
# # val_files = df[train_size:train_size + val_size]
# # test_files = df[train_size + val_size:]


train_texts, test_texts, train_labels, test_labels = train_test_split(df['Review'], df['Sentiment'], test_size=0.3, random_state=0)

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.25, random_state=0)


def preprocess(docs):
    return [underthesea.word_tokenize(doc.lower()) for doc in docs]


def get_vocabularies(tokenized_docs):
    vocabs = set()
    for doc in tokenized_docs:
        vocabs.update(doc)
    return vocabs

def identity_tokenizer(text):
    return text

# def create_vector(tokenized_docs):
    

# tokenized_docs = preprocess(train_texts[:SIZE_DEMO].astype('U'))
# tokenized_docs_val = preprocess(val_texts[:SIZE_DEMO].astype('U'))
# tokenized_docs_test = preprocess(test_texts[:SIZE_DEMO].astype('U'))
# vocabs = get_vocabularies(tokenized_docs)
# print(tokenized_docs)
tokenized_docs = train_texts[:SIZE_DEMO].astype('U')
tokenized_docs_val = val_texts[:SIZE_DEMO].astype('U')
tokenized_docs_test = test_texts[:SIZE_DEMO].astype('U')
    
        


# tfidf = TfidfVectorizer(analyzer="word", tokenizer=identity_tokenizer, token_pattern=None, lowercase=False)
tfidf = TfidfVectorizer(analyzer="word", tokenizer=underthesea.word_tokenize, token_pattern=None, lowercase=False)
# print(tokenized_docs[:100])
X_train = tfidf.fit_transform(tokenized_docs)
# print(X_train)
X_train.shape
y_train = train_labels[:SIZE_DEMO]
# X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2,
#                                                                                 random_state=0)
clf = LinearSVC().fit(X_train, y_train)
# clf = tree.DecisionTreeClassifier().fit(X_train, y_train)
# clf = MultinomialNB().fit(X_train, y_train)
# clf = RandomForestClassifier().fit(X_train, y_train)


def val_process(tfidf, clf):
    X_val = tfidf.transform(tokenized_docs_val)
    val_predictions = clf.predict(X_val)
    val_accuracy = accuracy_score(val_labels[:SIZE_DEMO], val_predictions)
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(classification_report(val_labels[:SIZE_DEMO], val_predictions))
    
    # docs_new = ['God is love', 'OpenGL on the GPU is fast']
    # X_new_tfidf = tfidf.transform(docs_new)
    
    # y_pred = clf.predict(X_new_tfidf)
    
    # for doc, category in zip(docs_new, y_pred):
    #     print('%r => %s' % (doc, category))
        
    # X_new_tfidf = tfidf.transform(test_texts)
    
    # predicted = clf.predict(X_new_tfidf)
    
    # np.mean(predicted == test_labels)
    
    param_grid = {
        'C': [0.1, 1, 10, 100, 1000],
        # 'max_iter': [1000, 5000, 10000],
    }
    
    # tree_param = {'criterion':['gini','entropy'],'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]}
    
    # parameters = {  
    #     'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)  
    # }  
    
    # parameters = {  
    #     'max_depth':[3,5,10,None],
    #               'n_estimators':[10,100,200],
    #               'max_features':[1,3,5,7],
    #               'min_samples_leaf':[1,2,3],
    #               'min_samples_split':[1,2,3]
    # }  
    
    
    # Initialize GridSearchCV
    grid_search = GridSearchCV(LinearSVC(), param_grid, cv=3, refit = True, scoring='accuracy')
    
    # grid_search = GridSearchCV(tree.DecisionTreeClassifier(), tree_param, cv=3, refit = True, scoring='accuracy')
    
    # grid_search = GridSearchCV(MultinomialNB(), parameters, cv=3, refit = True, scoring='accuracy')
    
    # grid_search = GridSearchCV(RandomForestClassifier(), parameters, cv=3, refit = True, scoring='accuracy')
    
    # Fit the grid search
    grid_search.fit(X_train, train_labels[:SIZE_DEMO])
    
    # Best parameters
    best_params = grid_search.best_params_
    print(f"Best Parameters: {best_params}")
    
    # Evaluate best model on validation set
    best_model = grid_search.best_estimator_
    val_predictions = best_model.predict(X_val)
    val_accuracy = accuracy_score(val_labels[:SIZE_DEMO], val_predictions)
    print(f"Validation Accuracy with Best Model: {val_accuracy:.4f}")
    print(classification_report(val_labels[:SIZE_DEMO], val_predictions))


val_process(tfidf, clf)


def train_demo(df_train, tfidf, clf):
    for index, row in df_train.iterrows():
        X_val = tfidf.transform([row['Review']])
        row['Sentiment'] = str(clf.predict(X_val)).strip('[]\'')
        print(f"{row['Review']}=>{row['Sentiment']}")


# train_demo(df_train, tfidf, clf)

D:\jupyter_proj\venv\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Validation Accuracy: 0.8534
              precision    recall  f1-score   support

    negative       0.85      0.87      0.86      2313
     neutral       0.70      0.36      0.48        97
    positive       0.86      0.86      0.86      2590

    accuracy                           0.85      5000
   macro avg       0.80      0.70      0.73      5000
weighted avg       0.85      0.85      0.85      5000



D:\jupyter_proj\venv\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\jupyter_proj\venv\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\jupyter_proj\venv\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\jupyter_proj\venv\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\jupyter_proj\venv\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWar

Best Parameters: {'C': 0.1}
Validation Accuracy with Best Model: 0.8486
              precision    recall  f1-score   support

    negative       0.85      0.86      0.86      2313
     neutral       1.00      0.05      0.10        97
    positive       0.85      0.86      0.86      2590

    accuracy                           0.85      5000
   macro avg       0.90      0.59      0.60      5000
weighted avg       0.85      0.85      0.84      5000



D:\jupyter_proj\venv\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
D:\jupyter_proj\venv\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
